In [1]:
import numpy as np
import pandas as pd

In [3]:
data= pd.read_csv("./train_NIR5Yl1.csv")

In [4]:
data.head()

,ID,Tag,Reputation,Answers,Username,Views,Upvotes
0,52664,a,3942.0,2.0,155623,7855.0,42.0
1,327662,a,26046.0,12.0,21781,55801.0,1175.0
2,468453,c,1358.0,4.0,56177,8067.0,60.0
3,96996,a,264.0,3.0,168793,27064.0,9.0
4,131465,c,4271.0,4.0,112223,13986.0,83.0


In [6]:
data.drop("ID",axis =1,inplace=True)

In [7]:
data.head()

,Tag,Reputation,Answers,Username,Views,Upvotes
0,a,3942.0,2.0,155623,7855.0,42.0
1,a,26046.0,12.0,21781,55801.0,1175.0
2,c,1358.0,4.0,56177,8067.0,60.0
3,a,264.0,3.0,168793,27064.0,9.0
4,c,4271.0,4.0,112223,13986.0,83.0


In [10]:
data_dummies=pd.get_dummies(data["Tag"])
data.head()

,Tag,Reputation,Answers,Username,Views,Upvotes
0,a,3942.0,2.0,155623,7855.0,42.0
1,a,26046.0,12.0,21781,55801.0,1175.0
2,c,1358.0,4.0,56177,8067.0,60.0
3,a,264.0,3.0,168793,27064.0,9.0
4,c,4271.0,4.0,112223,13986.0,83.0


In [11]:
data = pd.concat([data,pd.get_dummies(data['Tag'], drop_first=True)], axis=1); data.head()

,Tag,Reputation,Answers,Username,Views,Upvotes,c,h,i,j,o,p,r,s,x
0,a,3942.0,2.0,155623,7855.0,42.0,0,0,0,0,0,0,0,0,0
1,a,26046.0,12.0,21781,55801.0,1175.0,0,0,0,0,0,0,0,0,0
2,c,1358.0,4.0,56177,8067.0,60.0,1,0,0,0,0,0,0,0,0
3,a,264.0,3.0,168793,27064.0,9.0,0,0,0,0,0,0,0,0,0
4,c,4271.0,4.0,112223,13986.0,83.0,1,0,0,0,0,0,0,0,0


In [13]:
data.drop("Tag",axis=1,inplace=True)

In [14]:
data.head()

,Reputation,Answers,Username,Views,Upvotes,c,h,i,j,o,p,r,s,x
0,3942.0,2.0,155623,7855.0,42.0,0,0,0,0,0,0,0,0,0
1,26046.0,12.0,21781,55801.0,1175.0,0,0,0,0,0,0,0,0,0
2,1358.0,4.0,56177,8067.0,60.0,1,0,0,0,0,0,0,0,0
3,264.0,3.0,168793,27064.0,9.0,0,0,0,0,0,0,0,0,0
4,4271.0,4.0,112223,13986.0,83.0,1,0,0,0,0,0,0,0,0


In [15]:
from sklearn.model_selection import train_test_split

In [16]:
X = data.drop('Upvotes', axis=1)
y = data['Upvotes']

In [55]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.1,random_state=69)

In [56]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(297040, 13)
(297040,)
(33005, 13)
(33005,)


In [57]:
import xgboost as xgb

xg_reg=xgb.XGBRegressor(objective='reg:linear',n_estimators=10,seed=150)
xg_reg.fit(X_train,y_train)
preds=xg_reg.predict(X_test)

In [58]:
from sklearn.metrics import mean_squared_error
rmse=np.sqrt(mean_squared_error(y_test,preds))

In [59]:
print("RMSE : %f"% (rmse))

RMSE : 2276.641399


In [61]:
#tuning and boosting of xg boost

DM_train = xgb.DMatrix(data=X_train,label=y_train)
DM_test = xgb.DMatrix(data=X_test,label=y_test)
params = {"booster":"gblinear","objective":"reg:linear"}
xg_reg=xgb.train(params=params,dtrain=DM_train,num_boost_round=10)

preds_2=xg_reg.predict(DM_test)

rmse_2=np.sqrt(mean_squared_error(y_test,preds_2))

print("new RMSE= %f" %rmse_2)


new RMSE= 3299.232167


In [62]:
rmse

2276.6413991351656

In [63]:
rmse_2

3299.2321671460245

### Trying to apply grid search cv

# Attention !!!
#### i have just tried a bit more tuning....
initially i took num_boost_round=200 so it didn't run on my pc,so i reduced it to 2 then only rmse was 2368.but when i just increased to 20 th result is here!
So, try to run it at 200 rounds on gpu or even more. Also, try to change the nfolds.

In [67]:
review_data_matrix=xgb.DMatrix(data=X,label=y)

untuned_params = {"objective":"reg:linear"}

tuned_cv_rmse = xgb.cv(dtrain = review_data_matrix,params=untuned_params, nfold=5, num_boost_round=20, metrics="rmse",as_pandas=True, seed=123)

print(type(tuned_cv_rmse))

print("Tuned rmse: %f" %((tuned_cv_rmse["test-rmse-mean"]).tail(1)))


<class 'pandas.core.frame.DataFrame'>
Tuned rmse: 1411.574512


## trying more tuning methods

I didn't found this method benificial, so just go through it and if possible try hyperparameter tuning.that's it

In [69]:
tuned_params={"objective":"reg:linear",'colsample_bytree':0.3,'learning_rate':0.1,'max_depth':5}

new_tuned_cv_rmse = xgb.cv(dtrain=review_data_matrix, params=tuned_params, nfold=5, num_boost_round=20, metrics="rmse",as_pandas=True, seed=123)

print("Tuned rmse: %f" %((new_tuned_cv_rmse["test-rmse-mean"]).tail(1)))


Tuned rmse: 3126.445557
